In [1]:
!pip install psycopg2-binary

In [3]:
#!pip install psycopg2

In [4]:
import psycopg2 as pg

In [6]:
host="localhost"
database="ecommerce"
user="postgres"
password="test"
port = "5432"

In [23]:
import pandas as pd

conn = pg.connect(
            host=host,
            database=database,
            user=user,
            password=password,
            port = port
          )

cursor = conn.cursor()
cursor.execute("""
select * 
from orders o
join web_events w
on o.account_id = w.account_id
""")
rows = cursor.fetchall()
df = pd.DataFrame(rows)
header = [i[0] for i in cursor.description]
df.columns = header
#print(df.head(10))
cursor.close()
conn.close()
df.head(10)

,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd,id,account_id,occurred_at,channel
0,4318,1001,2016-11-25,485,543,177,1205,2420.15,4067.07,1437.24,7924.46,1,1001,2015-10-06,direct
1,4317,1001,2016-09-26,507,614,226,1347,2529.93,4598.86,1835.12,8963.91,1,1001,2015-10-06,direct
2,4316,1001,2016-08-28,557,572,255,1384,2779.43,4284.28,2070.60,9134.31,1,1001,2015-10-06,direct
3,4315,1001,2016-07-30,457,532,249,1238,2280.43,3984.68,2021.88,8286.99,1,1001,2015-10-06,direct
4,4314,1001,2016-05-31,531,603,209,1343,2649.69,4516.47,1697.08,8863.24,1,1001,2015-10-06,direct
5,4313,1001,2016-05-01,483,570,201,1254,2410.17,4269.30,1632.12,8311.59,1,1001,2015-10-06,direct
6,4312,1001,2016-04-01,497,618,152,1267,2480.03,4628.82,1234.24,8343.09,1,1001,2015-10-06,direct
7,4311,1001,2016-03-02,498,605,204,1307,2485.02,4531.45,1656.48,8672.95,1,1001,2015-10-06,direct
8,4310,1001,2016-02-01,473,595,212,1280,2360.27,4456.55,1721.44,8538.26,1,1001,2015-10-06,direct
9,4309,1001,2016-01-02,566,645,194,1405,2824.34,4831.05,1575.28,9230.67,1,1001,2015-10-06,direct


In [24]:
df.groupby("channel")["total"].sum().sort_values(ascending=False)

channel
direct      102408010
facebook     16682255
organic      16630210
adwords      15891377
twitter       8264526
banner        7916299
Name: total, dtype: object

In [57]:
def fetch_data(host, database, user, password, port, command):
    try:
        conn = pg.connect(
               host=host,
               database=database,
               user=user,
               password=password,
               port = port
              )
        cursor = conn.cursor()
        cursor.execute(command)
        rows = cursor.fetchall()
        df = pd.DataFrame(rows)
        header = [i[0] for i in cursor.description]
        df.columns = header
        return df
    
    except Exception as error:
        return "Connection error"
    finally:
        if cursor is None:
            cursor.close()
        if conn is None:
            conn.close()

In [46]:
def channel_with_sales(df):
    sales_channel = df.groupby("channel")["total"].sum().sort_values(ascending=False)
    return sales_channel

In [36]:
def accountid_sales(df):
    acc_sales = df.groupby("account_id")["total"].sum().sort_values(ascending=False)
    return acc_sales

In [62]:
host="localhost"
database="ecommerce"
user="postgres"
password="test"
port = "5432"
command = """
select * 
from orders o
join web_events w
on o.account_id = w.account_id
"""

command2 = """
select o.account_id, o.occurred_at, w.channel, o.total 
from orders o
join web_events w
on o.account_id = w.account_id
"""

command3 = """
select a.name, a.website, o.occurred_at, o.total
from accounts a
join orders o
on a.id = o.account_id
"""

In [38]:
df_data = fetch_data(host, database, user, password, port, command2)
df_data.head()

,account_id,occurred_at,channel,total
0,1001,2016-11-25,direct,1205
1,1001,2016-09-26,direct,1347
2,1001,2016-08-28,direct,1384
3,1001,2016-07-30,direct,1238
4,1001,2016-05-31,direct,1343


In [39]:
sales_channel = channel_with_sales(df_data)
sales_channel

channel
direct      102408010
facebook     16682255
organic      16630210
adwords      15891377
twitter       8264526
banner        7916299
Name: total, dtype: object

In [42]:
acc_df = accountid_sales(df_data)
acc_df.head()

account_id
4211    5020490
4151    4087418
4161    3982750
1871    3367210
3411    3219208
Name: total, dtype: object

In [64]:
df_r = fetch_data(host, database, user, password, port, command3)
df_r.head()

,name,website,occurred_at,total
0,Walmart,www.walmart.com,2015-10-06,169
1,Walmart,www.walmart.com,2015-11-05,288
2,Walmart,www.walmart.com,2015-12-04,132
3,Walmart,www.walmart.com,2016-01-02,176
4,Walmart,www.walmart.com,2016-02-01,165
